In [30]:
import numpy as np
import pandas as pd 
from skimage.data import imread
from skimage import draw
import matplotlib.pyplot as plt
import os
import pdb

In [31]:
masks = pd.read_csv('../../data/train_ship_segmentations.csv')
boxes = pd.read_csv('../../data/train_ship_boxes.csv',index_col=0)
boxes.head(5)

,ImageId,EncodedBoxes
0,00003e153.jpg,NaN
1,000155de5.jpg,"482.5 395.5 101.04 32.06 -1.54110209,"
2,00021ddc3.jpg,"746.5 149.0 36.06 10.82 -0.98279372,760.0 129...."
3,0002756f7.jpg,"48.5 341.5 24.60 6.71 0.46364761,53.5 334.0 24..."
4,0002d0f32.jpg,NaN
5,000303d4d.jpg,NaN
6,00031f145.jpg,"138.0 457.0 22.80 17.20 0.90975316,"
7,00053c6ba.jpg,"131.0 743.5 13.34 11.18 1.34399748,"
8,00057a50d.jpg,NaN
9,0005d01c8.jpg,"668.0 154.0 184.29 32.02 -0.89690284,286.0 515..."


In [32]:
train = os.listdir('../../data/train')
print(len(train))

104070


In [33]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [55]:
num_ships = []
R = []
C = []
L1 = []
L2 = []
theta = []
box_image_ids = []
all_image_ids = []
num_images = len(train)
for i in range(num_images):
    if i % 1000 == 0:
        print(i)
    box = boxes.iloc[i]['EncodedBoxes']
    ImageId = boxes.iloc[i]['ImageId'] 
    all_image_ids.append(ImageId)
    if box is np.nan:
        num_ships.append(0)
    else:
        img = imread('../../data/train/' + ImageId)
        img_masks = masks.loc[masks['ImageId'] == ImageId, 'EncodedPixels'].tolist()

        box_strings = box.split(',')
        num_ships.append(len(box_strings)-1)
        #pdb.set_trace()

        for j,box_string in enumerate(box_strings[:-1]):
            #pdb.set_trace()    
            mask = rle_decode(img_masks[j])
            box_vec = [float(x) for x in box_string.split()]
            theta_i = box_vec[4]
            L1_i = box_vec[2]
            L2_i = box_vec[3]
            midpoint = box_vec[0:2]
            C.append(midpoint[0])
            R.append(midpoint[1])
            L1.append(L1_i)
            L2.append(L2_i)
            theta.append(theta_i)
            box_image_ids.append(ImageId)
            if False:
                # construct rotated box for sanity check
                R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
                box = 0.5*np.array([[L1,L2],[L1,-L2],[-L1,-L2],[-L1,L2]]).T
                box_rot = np.dot(R,box)
                rr2, cc2 = draw.polygon_perimeter(midpoint[0]+box_rot[0,:],midpoint[1]+box_rot[1,:],
                                                  shape=(768,768),clip=True)

                img[rr2,cc2] = 255
                mask[rr2,cc2] = 2

                fig, axarr = plt.subplots(1,2,figsize=(12, 6))
                axarr[0].axis('off')
                axarr[0].imshow(img)
                axarr[0].set_ylim([max(rr2)+2, min(rr2)-2])
                axarr[0].set_xlim([max(cc2)+2, min(cc2)-2])
                axarr[1].axis('off')
                axarr[1].imshow(mask)
                axarr[1].set_ylim([max(rr2)+2, min(rr2)-2])
                axarr[1].set_xlim([max(cc2)+2, min(cc2)-2])
                plt.show()
                pdb.set_trace()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000


In [56]:
L1 = np.array(L1)
L2 = np.array(L2)
theta = np.array(theta)
C0 = np.array(C)
R0 = np.array(R)
num_ships = np.array(num_ships)


In [58]:
data_out = {'box_image_ids': box_image_ids,
           'L1': L1,
           'L2': L2,
           'theta': theta,
           'C0': C0,
           'R0': R0,
           'all_image_ids': all_image_ids,
           'num_ships': num_ships}

In [59]:
import pickle
outfile = '../../data/box_data.pickle'

with open(outfile, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data_out, f, pickle.HIGHEST_PROTOCOL)

In [60]:
if False:
    outfile = '../../data/box_data.npz'
    np.savez(outfile,L1=L1,L2=L2,theta=theta,C0=C0,R0=R0,
         num_ships=num_ships,image_ids=image_ids,all_image_ids=all_image_ids)